In [1]:
#DATA SETUP (Preprocessing & Augmentation)
import os
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D, Dropout
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
import numpy as np
from sklearn.metrics import classification_report, confusion_matrix

#Define Constants 
IMAGE_SIZE = (224, 224)
BATCH_SIZE = 32
NUM_CLASSES = 11  
FOLDER_NAME = 'images.cv_jzk6llhf18tm3k0kyttxz'
DATA_DIR = rf"C:\Users\sarit\OneDrive\Desktop\multiclass_fish\Dataset\{FOLDER_NAME}\data"
# Initialize Data Generators
train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=20,
    zoom_range=0.2,
    horizontal_flip=True,
    validation_split=0.2 # Reserve 20% for validation
)
test_datagen = ImageDataGenerator(rescale=1./255)

# Create Data Flow from Directory
train_generator = train_datagen.flow_from_directory(
    os.path.join(DATA_DIR,'train'),
    target_size=IMAGE_SIZE,
    batch_size=BATCH_SIZE,
    class_mode='categorical',
    subset='training',
    seed=42
)
validation_generator = train_datagen.flow_from_directory(
    os.path.join(DATA_DIR,'train'),
    target_size=IMAGE_SIZE,
    batch_size=BATCH_SIZE,
    class_mode='categorical',
    subset='validation',
    seed=42
)
test_generator = test_datagen.flow_from_directory(
    os.path.join(DATA_DIR,'test'),
    target_size=IMAGE_SIZE,
    batch_size=BATCH_SIZE,
    class_mode='categorical',
    shuffle=False
)




Found 4984 images belonging to 11 classes.
Found 1241 images belonging to 11 classes.
Found 3187 images belonging to 11 classes.


In [7]:
print(os.listdir(r"C:\Users\sarit\OneDrive\Desktop\multiclass_fish\Dataset"))


['images.cv_jzk6llhf18tm3k0kyttxz']


In [6]:
print("Class indices:", train_generator.class_indices)
print("Total classes:", train_generator.num_classes)


Class indices: {'animal fish': 0, 'animal fish bass': 1, 'fish sea_food black_sea_sprat': 2, 'fish sea_food gilt_head_bream': 3, 'fish sea_food hourse_mackerel': 4, 'fish sea_food red_mullet': 5, 'fish sea_food red_sea_bream': 6, 'fish sea_food sea_bass': 7, 'fish sea_food shrimp': 8, 'fish sea_food striped_red_mullet': 9, 'fish sea_food trout': 10}
Total classes: 11


In [ ]:
# MOBILENETV2 Transfer Learning Model Setup
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras.layers import GlobalAveragePooling2D, Dense, Dropout 
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
# Assuming NUM_CLASSES and BATCH_SIZE are defined globally


# Load Pre-trained Base Model (MobileNetV2)
base_model_mobilenet = MobileNetV2( 
    input_shape=(224, 224, 3),
    include_top=False, 
    weights='imagenet'
)

# Freeze the Base Layers
base_model_mobilenet.trainable = False 

x = base_model_mobilenet.output
x = GlobalAveragePooling2D()(x)
x = Dense(1024, activation='relu')(x)
x = Dropout(0.5)(x)
predictions = Dense(NUM_CLASSES, activation='softmax')(x)

# Create the full model
model_mobilenet = Model(inputs=base_model_mobilenet.input, outputs=predictions)


# Compile the Model
model_mobilenet.compile(
    optimizer=Adam(learning_rate=0.0001),
    loss='categorical_crossentropy',
    metrics=['accuracy']
)

# Train the Model
print("\n--- Starting MobileNetV2 Model Training ---")
history_mobilenet = model_mobilenet.fit( 
    steps_per_epoch=train_generator.samples // BATCH_SIZE,
    validation_data=validation_generator,
    validation_steps=validation_generator.samples // BATCH_SIZE,
    epochs=10
)

# Save the Trained Model
MODEL_SAVE_PATH_MOBILENET = 'mobilenetv2_classifier_model.h5' 
model_mobilenet.save(MODEL_SAVE_PATH_MOBILENET)
print(f"\nModel saved successfully as: {MODEL_SAVE_PATH_MOBILENET}")


#Final Test Set Evaluation
print("\nFinal Test Set Evaluation")
test_results_mobilenet = model_mobilenet.evaluate(
    test_generator,
    steps=test_generator.samples // BATCH_SIZE
)

# test_results will contain [loss, accuracy]
test_loss_mobilenet = test_results_mobilenet[0]
test_accuracy_mobilenet = test_results_mobilenet[1]

print(f"MobileNetV2 Final Test Loss: {test_loss_mobilenet:.4f}")
print(f"MobileNetV2 Final Test Accuracy: {test_accuracy_mobilenet:.4f}")


--- Starting MobileNetV2 Model Training ---
Epoch 1/10
155/155 ━━━━━━━━━━━━━━━━━━━━ 205s 1s/step - accuracy: 0.7264 - loss: 0.9037 - val_accuracy: 0.9490 - val_loss: 0.2799
Epoch 2/10
  1/155 ━━━━━━━━━━━━━━━━━━━━ 1:05 422ms/step - accuracy: 0.9375 - loss: 0.4533

c:\Users\sarit\OneDrive\Desktop\multiclass_fish\my_env\lib\site-packages\keras\src\trainers\epoch_iterator.py:116: UserWarning: Your input ran out of data; interrupting training. Make sure that your dataset or generator can generate at least `steps_per_epoch * epochs` batches. You may need to use the `.repeat()` function when building your dataset.
  self._interrupted_warning()


155/155 ━━━━━━━━━━━━━━━━━━━━ 26s 164ms/step - accuracy: 0.9375 - loss: 0.4533 - val_accuracy: 0.9523 - val_loss: 0.2657
Epoch 3/10
155/155 ━━━━━━━━━━━━━━━━━━━━ 132s 851ms/step - accuracy: 0.9287 - loss: 0.2813 - val_accuracy: 0.9704 - val_loss: 0.1442
Epoch 4/10
155/155 ━━━━━━━━━━━━━━━━━━━━ 25s 161ms/step - accuracy: 0.8750 - loss: 0.3588 - val_accuracy: 0.9688 - val_loss: 0.1441
Epoch 5/10
155/155 ━━━━━━━━━━━━━━━━━━━━ 125s 808ms/step - accuracy: 0.9546 - loss: 0.1769 - val_accuracy: 0.9737 - val_loss: 0.1092
Epoch 6/10
155/155 ━━━━━━━━━━━━━━━━━━━━ 25s 161ms/step - accuracy: 1.0000 - loss: 0.1040 - val_accuracy: 0.9778 - val_loss: 0.1060
Epoch 7/10
155/155 ━━━━━━━━━━━━━━━━━━━━ 171s 1s/step - accuracy: 0.9681 - loss: 0.1209 - val_accuracy: 0.9893 - val_loss: 0.0691
Epoch 8/10
155/155 ━━━━━━━━━━━━━━━━━━━━ 81s 512ms/step - accuracy: 0.9375 - loss: 0.1806 - val_accuracy: 0.9819 - val_loss: 0.0759
Epoch 9/10
155/155 ━━━━━━━━━━━━━━━━━━━━ 471s 3s/step - accuracy: 0.9729 - loss: 0.1012 - val_a


Model saved successfully as: mobilenetv2_classifier_model.h5

Final Test Set Evaluation
99/99 ━━━━━━━━━━━━━━━━━━━━ 194s 2s/step - accuracy: 0.9820 - loss: 0.0674
MobileNetV2 Final Test Loss: 0.0674
MobileNetV2 Final Test Accuracy: 0.9820


In [ ]:
# VGG16
from tensorflow.keras.applications import VGG16 
from tensorflow.keras.layers import GlobalAveragePooling2D, Dense, Dropout 

# Load Pre-trained Base Model (VGG16)
base_model = VGG16( 
    input_shape=(224, 224, 3),
    include_top=False,
    weights='imagenet'
)


base_model.trainable = False 


x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dense(1024, activation='relu')(x)
x = Dropout(0.5)(x)
predictions = Dense(NUM_CLASSES, activation='softmax')(x)

# Create the full model
model_vgg = Model(inputs=base_model.input, outputs=predictions) 

# Compile the Model
model_vgg.compile(
    optimizer=Adam(learning_rate=0.0001),
    loss='categorical_crossentropy',
    metrics=['accuracy']
)

# Train the Model
print("\n--- Starting VGG16 Model Training ---")
history_vgg = model_vgg.fit(
    train_generator,
    steps_per_epoch=train_generator.samples // BATCH_SIZE,
    validation_data=validation_generator,
    validation_steps=validation_generator.samples // BATCH_SIZE,
    epochs=10
)

# Save the Trained Model
MODEL_SAVE_PATH_VGG = 'vgg16_classifier_model.h5' 
model_vgg.save(MODEL_SAVE_PATH_VGG)
print(f"\nModel saved successfully as: {MODEL_SAVE_PATH_VGG}")


--- Starting VGG16 Model Training ---
Epoch 1/10
155/155 ━━━━━━━━━━━━━━━━━━━━ 1783s 12s/step - accuracy: 0.2668 - loss: 2.1133 - val_accuracy: 0.5477 - val_loss: 1.8049
Epoch 2/10
155/155 ━━━━━━━━━━━━━━━━━━━━ 303s 2s/step - accuracy: 0.4062 - loss: 1.7605 - val_accuracy: 0.5312 - val_loss: 1.8009
Epoch 3/10
155/155 ━━━━━━━━━━━━━━━━━━━━ 1333s 9s/step - accuracy: 0.4471 - loss: 1.7234 - val_accuracy: 0.6431 - val_loss: 1.5396
Epoch 4/10
155/155 ━━━━━━━━━━━━━━━━━━━━ 246s 2s/step - accuracy: 0.6250 - loss: 1.6700 - val_accuracy: 0.6398 - val_loss: 1.5450
Epoch 5/10
155/155 ━━━━━━━━━━━━━━━━━━━━ 1333s 9s/step - accuracy: 0.5749 - loss: 1.4800 - val_accuracy: 0.7031 - val_loss: 1.3318
Epoch 6/10
155/155 ━━━━━━━━━━━━━━━━━━━━ 253s 2s/step - accuracy: 0.7500 - loss: 1.2693 - val_accuracy: 0.6974 - val_loss: 1.3386
Epoch 7/10
155/155 ━━━━━━━━━━━━━━━━━━━━ 1219s 8s/step - accuracy: 0.6547 - loss: 1.2885 - val_accuracy: 0.7549 - val_loss: 1.1615
Epoch 8/10
155/155 ━━━━━━━━━━━━━━━━━━━━ 240s 2s/step 


Model saved successfully as: vgg16_classifier_model.h5


In [ ]:
# RESNET50 Transfer Learning Model 

#Model Imports
from tensorflow.keras.applications import ResNet50 
from tensorflow.keras.layers import GlobalAveragePooling2D, Dense, Dropout 
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam



# Load Pre-trained Base Model
base_model_resnet = ResNet50( 
    input_shape=(224, 224, 3),
    include_top=False,
    weights='imagenet'
)

# Freeze the Base Layers
base_model_resnet.trainable = False 


# Add Custom Classification Head 
x = base_model_resnet.output
x = GlobalAveragePooling2D()(x)
x = Dense(1024, activation='relu')(x)
x = Dropout(0.5)(x)
predictions = Dense(NUM_CLASSES, activation='softmax')(x)

# Create the full model
model_resnet = Model(inputs=base_model_resnet.input, outputs=predictions)


# Compile the Model
model_resnet.compile(
    optimizer=Adam(learning_rate=0.0001),
    loss='categorical_crossentropy',
    metrics=['accuracy']
)

# Train the Model
print("\n--- Starting ResNet50 Model Training ---")
history_resnet = model_resnet.fit(
    train_generator,
    steps_per_epoch=train_generator.samples // BATCH_SIZE,
    validation_data=validation_generator,
    validation_steps=validation_generator.samples // BATCH_SIZE,
    epochs=10
)

# Save the Trained Model
MODEL_SAVE_PATH_RESNET = 'resnet50_classifier_model.h5' 
model_resnet.save(MODEL_SAVE_PATH_RESNET)
print(f"\nModel saved successfully as: {MODEL_SAVE_PATH_RESNET}")

print("\nFinal Test Set Evaluation")
test_results = model_resnet.evaluate(
    test_generator,
    steps=test_generator.samples // BATCH_SIZE
)

test_loss_resnet = test_results[0]
test_accuracy_resnet = test_results[1]

print(f"ResNet50 Final Test Loss: {test_loss_resnet:.4f}")
print(f"ResNet50 Final Test Accuracy: {test_accuracy_resnet:.4f}")


94765736/94765736 ━━━━━━━━━━━━━━━━━━━━ 66s 1us/step

--- Starting ResNet50 Model Training ---
Epoch 1/10
155/155 ━━━━━━━━━━━━━━━━━━━━ 601s 4s/step - accuracy: 0.1422 - loss: 2.4585 - val_accuracy: 0.1793 - val_loss: 2.2220
Epoch 2/10
  1/155 ━━━━━━━━━━━━━━━━━━━━ 5:52 2s/step - accuracy: 0.1875 - loss: 2.1725

c:\Users\sarit\OneDrive\Desktop\multiclass_fish\my_env\lib\site-packages\keras\src\trainers\epoch_iterator.py:116: UserWarning: Your input ran out of data; interrupting training. Make sure that your dataset or generator can generate at least `steps_per_epoch * epochs` batches. You may need to use the `.repeat()` function when building your dataset.
  self._interrupted_warning()


155/155 ━━━━━━━━━━━━━━━━━━━━ 85s 537ms/step - accuracy: 0.1875 - loss: 2.1725 - val_accuracy: 0.1793 - val_loss: 2.2221
Epoch 3/10
155/155 ━━━━━━━━━━━━━━━━━━━━ 413s 3s/step - accuracy: 0.1799 - loss: 2.2803 - val_accuracy: 0.2245 - val_loss: 2.1838
Epoch 4/10
155/155 ━━━━━━━━━━━━━━━━━━━━ 81s 515ms/step - accuracy: 0.2500 - loss: 2.1031 - val_accuracy: 0.2278 - val_loss: 2.1808
Epoch 5/10
155/155 ━━━━━━━━━━━━━━━━━━━━ 455s 3s/step - accuracy: 0.2032 - loss: 2.1978 - val_accuracy: 0.1850 - val_loss: 2.1364
Epoch 6/10
155/155 ━━━━━━━━━━━━━━━━━━━━ 82s 517ms/step - accuracy: 0.0625 - loss: 2.3533 - val_accuracy: 0.1850 - val_loss: 2.1362
Epoch 7/10
155/155 ━━━━━━━━━━━━━━━━━━━━ 480s 3s/step - accuracy: 0.2197 - loss: 2.1569 - val_accuracy: 0.2368 - val_loss: 2.1026
Epoch 8/10
155/155 ━━━━━━━━━━━━━━━━━━━━ 100s 633ms/step - accuracy: 0.2500 - loss: 2.1002 - val_accuracy: 0.2327 - val_loss: 2.1141
Epoch 9/10
155/155 ━━━━━━━━━━━━━━━━━━━━ 509s 3s/step - accuracy: 0.2377 - loss: 2.1201 - val_accura


Model saved successfully as: resnet50_classifier_model.h5

Final Test Set Evaluation
99/99 ━━━━━━━━━━━━━━━━━━━━ 284s 3s/step - accuracy: 0.2595 - loss: 2.0710
ResNet50 Final Test Loss: 2.0710
ResNet50 Final Test Accuracy: 0.2595


In [ ]:
# INCEPTIONV3 Transfer Learning Model
from tensorflow.keras.applications import InceptionV3
from tensorflow.keras.layers import GlobalAveragePooling2D, Dense, Dropout 
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam

#InceptionV3 standard input size 
base_model_inception = InceptionV3(
    input_shape=(224, 224, 3),
    include_top=False,
    weights='imagenet'
)

base_model_inception.trainable = False 

x = base_model_inception.output
x = GlobalAveragePooling2D()(x)
x = Dense(1024, activation='relu')(x)
x = Dropout(0.5)(x)
predictions = Dense(NUM_CLASSES, activation='softmax')(x)

# Create the full model
model_inception = Model(inputs=base_model_inception.input, outputs=predictions) 

# Compile the Model
model_inception.compile(
    optimizer=Adam(learning_rate=0.0001),
    loss='categorical_crossentropy',
    metrics=['accuracy']
)

# Train the Model
print("\nStarting InceptionV3 Model Training")
history_inception = model_inception.fit(
    train_generator,
    steps_per_epoch=train_generator.samples // BATCH_SIZE,
    validation_data=validation_generator,
    validation_steps=validation_generator.samples // BATCH_SIZE,
    epochs=10
)

# Save the Trained Model
MODEL_SAVE_PATH_INCEPTION = 'inceptionv3_classifier_model.h5' 
model_inception.save(MODEL_SAVE_PATH_INCEPTION)
print(f"\nModel saved successfully as: {MODEL_SAVE_PATH_INCEPTION}")


#'model_inception' is trained model and 'test_generator' is data generator

print("\nFinal Test Set Evaluation")
test_results_inception = model_inception.evaluate(
    test_generator,
    steps=test_generator.samples // BATCH_SIZE
)

# test_results will contain [loss, accuracy]
test_loss_inception = test_results_inception[0]
test_accuracy_inception = test_results_inception[1]

print(f"InceptionV3 Final Test Loss: {test_loss_inception:.4f}")
print(f"InceptionV3 Final Test Accuracy: {test_accuracy_inception:.4f}")

87910968/87910968 ━━━━━━━━━━━━━━━━━━━━ 6s 0us/step

Starting InceptionV3 Model Training
Epoch 1/10
155/155 ━━━━━━━━━━━━━━━━━━━━ 354s 2s/step - accuracy: 0.6961 - loss: 0.9574 - val_accuracy: 0.9219 - val_loss: 0.3338
Epoch 2/10
155/155 ━━━━━━━━━━━━━━━━━━━━ 56s 353ms/step - accuracy: 0.8750 - loss: 0.2730 - val_accuracy: 0.9194 - val_loss: 0.3336
Epoch 3/10
155/155 ━━━━━━━━━━━━━━━━━━━━ 285s 2s/step - accuracy: 0.8835 - loss: 0.3619 - val_accuracy: 0.9391 - val_loss: 0.2324
Epoch 4/10
155/155 ━━━━━━━━━━━━━━━━━━━━ 56s 352ms/step - accuracy: 0.7500 - loss: 0.6357 - val_accuracy: 0.9449 - val_loss: 0.2111
Epoch 5/10
155/155 ━━━━━━━━━━━━━━━━━━━━ 284s 2s/step - accuracy: 0.9150 - loss: 0.2671 - val_accuracy: 0.9572 - val_loss: 0.1641
Epoch 6/10
155/155 ━━━━━━━━━━━━━━━━━━━━ 56s 352ms/step - accuracy: 0.9062 - loss: 0.2277 - val_accuracy: 0.9556 - val_loss: 0.1626
Epoch 7/10
155/155 ━━━━━━━━━━━━━━━━━━━━ 284s 2s/step - accuracy: 0.9307 - loss: 0.2112 - val_accuracy: 0.9696 - val_loss: 0.1206
Epo


Model saved successfully as: inceptionv3_classifier_model.h5

Final Test Set Evaluation
99/99 ━━━━━━━━━━━━━━━━━━━━ 154s 2s/step - accuracy: 0.9732 - loss: 0.0957
InceptionV3 Final Test Loss: 0.0957
InceptionV3 Final Test Accuracy: 0.9732


In [ ]:
# EFFICIENTNETB0 Transfer Learning Model Setup
from tensorflow.keras.applications import EfficientNetB0 
from tensorflow.keras.layers import GlobalAveragePooling2D, Dense, Dropout 
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam



# Load Pre-trained Base Model (EfficientNetB0)
base_model_effnet = EfficientNetB0( 
    input_shape=(224, 224, 3),
    include_top=False,
    weights='imagenet'
)

# Freeze the Base Layers
base_model_effnet.trainable = False 


# Add Custom Classification Head
x = base_model_effnet.output
x = GlobalAveragePooling2D()(x)
x = Dense(1024, activation='relu')(x)
x = Dropout(0.5)(x)
predictions = Dense(NUM_CLASSES, activation='softmax')(x)

# Create the full model
model_effnet = Model(inputs=base_model_effnet.input, outputs=predictions) 


# Compile the Model
model_effnet.compile(
    optimizer=Adam(learning_rate=0.0001),
    loss='categorical_crossentropy',
    metrics=['accuracy']
)

# Train the Model
print("\n--- Starting EfficientNetB0 Model Training ---")
history_effnet = model_effnet.fit( 
    train_generator,
    steps_per_epoch=train_generator.samples // BATCH_SIZE,
    validation_data=validation_generator,
    validation_steps=validation_generator.samples // BATCH_SIZE,
    epochs=10
)

# Save the Trained Model
MODEL_SAVE_PATH_EFFNET = 'efficientnetb0_classifier_model.h5' 
model_effnet.save(MODEL_SAVE_PATH_EFFNET)
print(f"\nModel saved successfully as: {MODEL_SAVE_PATH_EFFNET}")


print("\nFinal Test Set Evaluation")
test_results_effnet = model_effnet.evaluate(
    test_generator,
    steps=test_generator.samples // BATCH_SIZE
)

# test_results will contain [loss, accuracy]
test_loss_effnet = test_results_effnet[0]
test_accuracy_effnet = test_results_effnet[1]

print(f"EfficientNetB0 Final Test Loss: {test_loss_effnet:.4f}")
print(f"EfficientNetB0 Final Test Accuracy: {test_accuracy_effnet:.4f}")


16705208/16705208 ━━━━━━━━━━━━━━━━━━━━ 3s 0us/step

--- Starting EfficientNetB0 Model Training ---
Epoch 1/10
155/155 ━━━━━━━━━━━━━━━━━━━━ 160s 1s/step - accuracy: 0.1601 - loss: 2.3310 - val_accuracy: 0.1776 - val_loss: 2.3023
Epoch 2/10
155/155 ━━━━━━━━━━━━━━━━━━━━ 31s 199ms/step - accuracy: 0.2188 - loss: 2.3002 - val_accuracy: 0.1776 - val_loss: 2.2993
Epoch 3/10
155/155 ━━━━━━━━━━━━━━━━━━━━ 164s 1s/step - accuracy: 0.1692 - loss: 2.3244 - val_accuracy: 0.1768 - val_loss: 2.2958
Epoch 4/10
155/155 ━━━━━━━━━━━━━━━━━━━━ 33s 211ms/step - accuracy: 0.0625 - loss: 2.4094 - val_accuracy: 0.1735 - val_loss: 2.2982
Epoch 5/10
155/155 ━━━━━━━━━━━━━━━━━━━━ 165s 1s/step - accuracy: 0.1733 - loss: 2.3151 - val_accuracy: 0.1793 - val_loss: 2.2968
Epoch 6/10
155/155 ━━━━━━━━━━━━━━━━━━━━ 33s 211ms/step - accuracy: 0.1562 - loss: 2.4006 - val_accuracy: 0.1760 - val_loss: 2.2996
Epoch 7/10
155/155 ━━━━━━━━━━━━━━━━━━━━ 156s 1s/step - accuracy: 0.1755 - loss: 2.3088 - val_accuracy: 0.1768 - val_loss:


Model saved successfully as: efficientnetb0_classifier_model.h5

Final Test Set Evaluation
99/99 ━━━━━━━━━━━━━━━━━━━━ 65s 657ms/step - accuracy: 0.1641 - loss: 2.3045
EfficientNetB0 Final Test Loss: 2.3045
EfficientNetB0 Final Test Accuracy: 0.1641


In [ ]:


# MODEL EVALUATION
print("\n--- Model Evaluation on Test Set ---")
loss, accuracy = model_mobilenet.evaluate(test_generator, verbose=1)
print(f"Test Loss: {loss:.4f}")
print(f"Test Accuracy: {accuracy:.4f}")

# Generate Classification Report and Confusion Matrix
Y_pred = model_mobilenet.predict(test_generator)
y_pred = np.argmax(Y_pred, axis=1)
y_true = test_generator.classes
class_names = list(test_generator.class_indices.keys())

print("\n--- Classification Report ---")
print(classification_report(y_true, y_pred, target_names=class_names))


--- Model Evaluation on Test Set ---
100/100 ━━━━━━━━━━━━━━━━━━━━ 98s 981ms/step - accuracy: 0.9896 - loss: 0.0518
Test Loss: 0.0518
Test Accuracy: 0.9896
100/100 ━━━━━━━━━━━━━━━━━━━━ 66s 641ms/step

--- Classification Report ---
                                  precision    recall  f1-score   support

                     animal fish       0.98      1.00      0.99       520
                animal fish bass       1.00      0.15      0.27        13
   fish sea_food black_sea_sprat       1.00      0.98      0.99       298
   fish sea_food gilt_head_bream       0.99      0.98      0.99       305
   fish sea_food hourse_mackerel       1.00      0.99      1.00       286
        fish sea_food red_mullet       0.99      1.00      0.99       291
     fish sea_food red_sea_bream       0.98      1.00      0.99       273
          fish sea_food sea_bass       0.98      1.00      0.99       327
            fish sea_food shrimp       1.00      1.00      1.00       289
fish sea_food striped_red_mu

In [4]:
from tensorflow.keras.models import load_model
from sklearn.metrics import classification_report
import numpy as np

# Load the trained model
model_vgg = load_model(r"C:\Users\sarit\OneDrive\Desktop\multiclass_fish\vgg16_classifier_model.h5")

# Evaluate on test set
print("\n--- Model Evaluation on Test Set (VGG16) ---")
loss, accuracy = model_vgg.evaluate(test_generator, verbose=1)
print(f"VGG16 Test Loss: {loss:.4f}")
print(f"VGG16 Test Accuracy: {accuracy:.4f}")

# Predictions
Y_pred = model_vgg.predict(test_generator)
y_pred = np.argmax(Y_pred, axis=1)
y_true = test_generator.classes
class_names = list(test_generator.class_indices.keys())

print("\n--- Classification Report ---")
print(classification_report(y_true, y_pred, target_names=class_names))



--- Model Evaluation on Test Set (VGG16) ---
100/100 ━━━━━━━━━━━━━━━━━━━━ 823s 8s/step - accuracy: 0.7772 - loss: 0.9956
VGG16 Test Loss: 0.9956
VGG16 Test Accuracy: 0.7772
100/100 ━━━━━━━━━━━━━━━━━━━━ 789s 8s/step

--- Classification Report ---
                                  precision    recall  f1-score   support

                     animal fish       0.97      0.98      0.98       520
                animal fish bass       0.00      0.00      0.00        13
   fish sea_food black_sea_sprat       0.77      0.89      0.83       298
   fish sea_food gilt_head_bream       0.94      0.33      0.49       305
   fish sea_food hourse_mackerel       0.61      0.95      0.74       286
        fish sea_food red_mullet       0.65      0.55      0.59       291
     fish sea_food red_sea_bream       0.71      0.86      0.78       273
          fish sea_food sea_bass       0.89      0.70      0.79       327
            fish sea_food shrimp       0.72      0.99      0.83       289
fish sea_foo

c:\Users\sarit\OneDrive\Desktop\multiclass_fish\my_env\lib\site-packages\sklearn\metrics\_classification.py:1731: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", result.shape[0])
c:\Users\sarit\OneDrive\Desktop\multiclass_fish\my_env\lib\site-packages\sklearn\metrics\_classification.py:1731: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", result.shape[0])
c:\Users\sarit\OneDrive\Desktop\multiclass_fish\my_env\lib\site-packages\sklearn\metrics\_classification.py:1731: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(ave

In [ ]:
from tensorflow.keras.models import load_model
from sklearn.metrics import classification_report
import numpy as np

# Dictionary of model names and their respective file paths
model_paths = {
    "InceptionV3": r"C:\Users\sarit\OneDrive\Desktop\multiclass_fish\inceptionv3_classifier_model.h5",
    "ResNet50": r"C:\Users\sarit\OneDrive\Desktop\multiclass_fish\resnet50_classifier_model.h5",
    "EfficientNetB0": r"C:\Users\sarit\OneDrive\Desktop\multiclass_fish\efficientnetb0_classifier_model.h5"
}

# Loop through each model and evaluate
for model_name, model_path in model_paths.items():
    print(f"\nModel Evaluation on Test Set ({model_name}) ")
    
    # Load model
    model = load_model(model_path)

    # Evaluate performance
    loss, accuracy = model.evaluate(test_generator, verbose=1)
    print(f"{model_name} Test Loss: {loss:.4f}")
    print(f"{model_name} Test Accuracy: {accuracy:.4f}")

    # Generate predictions
    Y_pred = model.predict(test_generator)
    y_pred = np.argmax(Y_pred, axis=1)
    y_true = test_generator.classes
    class_names = list(test_generator.class_indices.keys())

    # Classification report
    print(f"\nClassification Report ({model_name})")
    print(classification_report(y_true, y_pred, target_names=class_names))



--- Model Evaluation on Test Set (InceptionV3) ---


100/100 ━━━━━━━━━━━━━━━━━━━━ 132s 1s/step - accuracy: 0.9733 - loss: 0.0952
InceptionV3 Test Loss: 0.0952
InceptionV3 Test Accuracy: 0.9733
100/100 ━━━━━━━━━━━━━━━━━━━━ 142s 1s/step

--- Classification Report (InceptionV3) ---
                                  precision    recall  f1-score   support

                     animal fish       0.97      0.99      0.98       520
                animal fish bass       1.00      0.15      0.27        13
   fish sea_food black_sea_sprat       0.98      0.99      0.99       298
   fish sea_food gilt_head_bream       0.98      0.97      0.98       305
   fish sea_food hourse_mackerel       0.96      0.98      0.97       286
        fish sea_food red_mullet       0.99      0.95      0.97       291
     fish sea_food red_sea_bream       0.99      0.98      0.99       273
          fish sea_food sea_bass       0.96      0.96      0.96       327
            fish sea_food shrimp       1.00      0.99      0.99       289
fish sea_food striped_red_mullet

100/100 ━━━━━━━━━━━━━━━━━━━━ 217s 2s/step - accuracy: 0.2611 - loss: 2.0704
ResNet50 Test Loss: 2.0704
ResNet50 Test Accuracy: 0.2611
100/100 ━━━━━━━━━━━━━━━━━━━━ 216s 2s/step

--- Classification Report (ResNet50) ---
                                  precision    recall  f1-score   support

                     animal fish       0.26      0.72      0.38       520
                animal fish bass       0.00      0.00      0.00        13
   fish sea_food black_sea_sprat       0.45      0.06      0.11       298
   fish sea_food gilt_head_bream       0.00      0.00      0.00       305
   fish sea_food hourse_mackerel       0.24      0.22      0.23       286
        fish sea_food red_mullet       0.26      0.70      0.38       291
     fish sea_food red_sea_bream       0.00      0.00      0.00       273
          fish sea_food sea_bass       0.75      0.01      0.02       327
            fish sea_food shrimp       0.00      0.00      0.00       289
fish sea_food striped_red_mullet       0.

c:\Users\sarit\OneDrive\Desktop\multiclass_fish\my_env\lib\site-packages\sklearn\metrics\_classification.py:1731: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", result.shape[0])
c:\Users\sarit\OneDrive\Desktop\multiclass_fish\my_env\lib\site-packages\sklearn\metrics\_classification.py:1731: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", result.shape[0])
c:\Users\sarit\OneDrive\Desktop\multiclass_fish\my_env\lib\site-packages\sklearn\metrics\_classification.py:1731: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(ave

100/100 ━━━━━━━━━━━━━━━━━━━━ 71s 680ms/step - accuracy: 0.1632 - loss: 2.3045
EfficientNetB0 Test Loss: 2.3045
EfficientNetB0 Test Accuracy: 0.1632
100/100 ━━━━━━━━━━━━━━━━━━━━ 71s 690ms/step

--- Classification Report (EfficientNetB0) ---
                                  precision    recall  f1-score   support

                     animal fish       0.16      1.00      0.28       520
                animal fish bass       0.00      0.00      0.00        13
   fish sea_food black_sea_sprat       0.00      0.00      0.00       298
   fish sea_food gilt_head_bream       0.00      0.00      0.00       305
   fish sea_food hourse_mackerel       0.00      0.00      0.00       286
        fish sea_food red_mullet       0.00      0.00      0.00       291
     fish sea_food red_sea_bream       0.00      0.00      0.00       273
          fish sea_food sea_bass       0.00      0.00      0.00       327
            fish sea_food shrimp       0.00      0.00      0.00       289
fish sea_food strip

c:\Users\sarit\OneDrive\Desktop\multiclass_fish\my_env\lib\site-packages\sklearn\metrics\_classification.py:1731: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", result.shape[0])
c:\Users\sarit\OneDrive\Desktop\multiclass_fish\my_env\lib\site-packages\sklearn\metrics\_classification.py:1731: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", result.shape[0])
c:\Users\sarit\OneDrive\Desktop\multiclass_fish\my_env\lib\site-packages\sklearn\metrics\_classification.py:1731: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(ave